# Agent Based Modelling
## Drake Assignment 4

Oh uh!  The unthinkable has occurred and a zombie outbreak has occurred on Square Island!  This island is 100 square miles and evenly populated by 100 citizens.  Can Square Island's once peaceful populace hold out for 10 days until help arrives?  Let's find out!

In [5]:
#First, let's load some packages, including mesa

import random
import mesa
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import Grid
from mesa.datacollection import DataCollector
from mesa.batchrunner import BatchRunner

In [10]:
#Below we define the agent aspects of the model
#The citizens of Square Island can have three states: Human, Zombie, or Dead

class CitizenCell(Agent):

    def __init__(self, model, pos):
     
        super().__init__(pos, model)
        self.pos = pos
        self.unique_id = pos
        self.condition = "Human"
        
        
#Two things can happen each day (timestep), zombies can infect neighboring humans and humans can defend themselves and 
#kill neighboring zombies
        
        
    def step(self):
        
        #Zombies infect their neighbors with a 50% probability
        if self.condition == "Zombie":
            neighbors = self.model.grid.get_neighbors(self.pos, moore=False)
            for neighbor in neighbors:
                if neighbor.condition == "Human" and random.random()>=.5:
                    neighbor.condition = "Zombie"
            self.condition = "Zombie"
            
        #Humans only defend themselves with a 10% probability    
        if self.condition == "Human":
            neighbors = self.model.grid.get_neighbors(self.pos, moore=False)
            for neighbor in neighbors:
                if neighbor.condition == "Zombie" and random.random()>=.9:
                    neighbor.condition = "Dead"
            self.condition = "Human"

In [11]:
# Next, we define 
class Outbreak(Model):
   
    def __init__(self, height, width, density):
        '''
        Create a new forest fire model.
        
        Args:
            height, width: The size of the grid to model
            density: What fraction of grid cells have a tree in them.
        '''
        # Initialize model parameters
        self.height = height
        self.width = width
        self.density = density
        
        # Set up model objects
        self.schedule = RandomActivation(self)
        self.grid = Grid(height, width, torus=False)
        self.dc = DataCollector({"Human": lambda m: self.count_type(m, "Human"),
                                "Zombie": lambda m: self.count_type(m, "Zombie"),
                                "Dead": lambda m: self.count_type(m, "Dead")})
        
        # Place a citizen in each cell with Prob = density
        for x in range(self.width):
            for y in range(self.height):
                if random.random() < self.density:
                    # Create a citizen
                    new_citizen = CitizenCell(self, (x, y))
                    # SInfect all humans in the first column.
                    if x == 0:
                        new_citizen.condition = "Zombie"
                    self.grid[y][x] = new_citizen
                    self.schedule.add(new_citizen)
        self.running = True
        
    def step(self):
        '''
        Advance the model by one step.
        '''
        self.schedule.step()
        self.dc.collect(self)
        # Halt if no more humans
        if self.count_type(self, "Human") == 0:
            self.running = False
    
    def count_type(model, citizen_condition):
        '''
        Helper method to count citizens in a given condition in a given model.
        '''
        count = 0
        for citizen in model.schedule.agents:
            if citizen.condition == citizen_condition:
                count += 1
        return count

In [12]:
#Now we populate our island and set the island's density parameter, which in this case is 1
outbreak = Outbreak(10, 10, 1)
for i in range(10):
    model.step()

In [13]:
#The outbreak has begun!  Let's run this model

outbreak.run_model()

TypeError: count_type() takes 2 positional arguments but 3 were given